# Анонимные функции

Функции, которые имеют название, **можно объявлять** только на уровне пакета. Но в Go с помощью литерала функции можно создавать и так называемые **анонимные функции** - функции, которые не имеют имени и которые можно определять в любом месте программы. Литерал функции похож на определение функции, только после ключевого слова func не указывается название функции.

Литерал функции позволяет нам определить функции только в том месте, где они используются. Пример анонимной функции:

In [2]:
import (
    "fmt"
    "strings"
)

str := strings.Map(func(r rune) rune { return r + 1 }, "HAL-9000")
fmt.Sprint(str)

IBM.:111


Одной из отличительных особенностей анонимных функций является то, что они **имеют доступ** ко всему лексическому блоку, в котором они определены, т.е. они могут обращаться к переменным, находящимся в функции, которая включает в себя анонимную функцию:

In [11]:
func squares() func() int {
    var x int
    return func() int {
        x++
        return x*x
    }
}

f := squares()
fmt.Println(f())
fmt.Println(f())
fmt.Println(f())
fmt.Println(f())
fmt.Println(f())

fmt.Println()

// Ещё один вызов squares() создаст уже другую перменную x, которую будет увеличивать вызов g().
g := squares()
fmt.Println(g())
fmt.Println(g())
fmt.Sprint(g())

1
4
9
16
25

1
4
9


Этот пример показывает, что значения типа функции(function values) кроме кода имеют и **состояние**. Анонимная внутренняя функция имеет доступ, а также может изменить локальную переменную заключающей её функции. Из-за этих скрытых ссылок на переменные мы и классифицируем функции как ссылочные типы и не можем сравнивать значения функций между собой. Значения функций реализуются с помощью метода называемого **замыканием**(closure) и часто значение типа функции называют замыканием.

Этот пример также демонстрирует ещё один случай, когда время жизни переменной не определяется её областью видимости: переменная x существует и после того как выполнилась функция squares, хотя она является локальной переменной(первый случай был, когда вернули указатель на локальную переменную; в Go такое можно делать).

## Ловушка переменных итерирования

Рассмотрим следующий код:

In [16]:
var deferredCalls []func()

for i := 0; i < 3; i++ {
    deferredCalls = append(deferredCalls, func() { fmt.Println(i) })
}

for _, call := range deferredCalls {
    call()
}

3
3
3


Мы можем увидеть довольно неожиданный на первый взгляд результат: выведены только тройки. Откуда взялись эти тройки? Здесь дело в следующем: при объявлении цикла for был создан лексический блок, в который входит и переменная итерирования i. При формировании списка отложенных вызовов deferredCalls все анонимные функции "подхватывают" одну и ту же переменную i - адресуемое хранилище данных, а не значение переменной в определённый момент времени. Переменная i обновляется с каждой итерацией и к концу цикла она равна 3. Поэтому и выводились тройки.

Чтобы вывести все значения i необходимо переписать код так:

In [15]:
var deferredCalls []func()

for i := 0; i < 3; i++ {
    iCopy := i
    deferredCalls = append(deferredCalls, func() { fmt.Println(iCopy) })
}

for _, call := range deferredCalls {
    call()
}

0
1
2


Теперь с каждой итерацией создаётся новая локальная переменная, которую и "подхватывают" анонимные функции.

Такую ошибку наиболее часто делают при работе с **go** или **defer**, которые могут отложить вызов функции до завершения выполнения цикла.